In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame (rdf)
rdf = pd.read_csv(r"filtered_output.csv")

# Convert the 'Timestamp' column to datetime format
rdf['Timestamp'] = pd.to_datetime(rdf['Timestamp'])

# Format the 'Timestamp' column to include only hours, minutes, and seconds ('%H:%M:%S')
rdf['Timestamp'] = rdf['Timestamp'].dt.strftime('%H:%M:%S')

# Group rdf by 'Timestamp' and aggregate each group as a list of values
rdf_grouped = rdf.groupby('Timestamp').agg(lambda x: x.tolist()).reset_index()

# Keep only the first pose for each second
#rdf_grouped['Pose'] = rdf_grouped['Pose'].apply(lambda x: x[0] if x else None)

# Set the 'Timestamp' column as the index
rdf_grouped.set_index('Timestamp', inplace=True)

# Save the resulting DataFrame (excluding the 'Time' column) to a new CSV file
rdf_grouped.to_csv("aggregated_data_signal.csv", header=True, columns=['Channel 1', 'Channel 2', 'Channel 3', 'Channel 4'])

# Display the resulting DataFrame (rdf_grouped)
print(rdf_grouped)


In [ ]:
import pandas as pd
from scipy.interpolate import CubicSpline

# Replace 'your_file.csv' with the actual path to your CSV file
file_path = 'aggregated_data_signal.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Columns to consider
channels = ['Channel 1', 'Channel 2', 'Channel 3', 'Channel 4']
# Find the maximum number of elements across all cells in specified columns
max_elements = max(df[channels].apply(lambda x: x.map(lambda y: len(eval(y)))).max())
print(max_elements)

for column in channels:
    print(f"Column: {column}")
    for index, cell_value in enumerate(df[column]):
        cell_value = eval(cell_value)
        # Generate indices for the expanded data
        expanded_indices = np.linspace(0, len(cell_value) - 1, max_elements)
        # Create a CubicSpline interpolation function
        cubic_spline = CubicSpline(np.arange(len(cell_value)), cell_value, bc_type='clamped', extrapolate=False)
        # Perform cubic spline interpolation on the expanded indices
        expanded_data = cubic_spline(expanded_indices)
        #print(f"  Row {index + 1}: {cell_value}")
        df.at[index, column] = expanded_data.tolist()

# Save the DataFrame to a new CSV file with interpolated values
df.to_csv('interpolated_file_signal.csv', index=False)


In [ ]:
import pandas as pd
import numpy as np
from scipy.interpolate import PchipInterpolator

# Replace 'your_file.csv' with the actual path to your CSV file
file_path = 'aggregated_data_signal.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Columns to consider
channels = ['Channel 1', 'Channel 2', 'Channel 3', 'Channel 4']

# Find the maximum number of elements across all cells in specified columns
#max_elements = max(df[channels].apply(lambda x: x.map(lambda y: len(eval(y)))).max())
max_elements = 200
print(max_elements)
for column in channels:
    print(f"Column: {column}")
    for index, cell_value in enumerate(df[column]):
        # Apply eval to each element (cell) within the Series
        cell_value = eval(cell_value)
        
        # Generate indices for the expanded data
        expanded_indices = np.linspace(0, len(cell_value) - 1, max_elements)
        
        # Create a PchipInterpolator interpolation function
        pchip_interp = PchipInterpolator(np.arange(len(cell_value)), cell_value)
        
        # Perform Pchip interpolation on the expanded indices
        expanded_data = pchip_interp(expanded_indices)
        
        # Update the DataFrame with interpolated values
        df.at[index, column] = expanded_data.tolist()

# Save the DataFrame to a new CSV file with interpolated values
df.to_csv('interpolated_file_signal.csv', index=False)


In [ ]:
import pandas as pd
import ast

def expand_chn_data(df):
    # Create an empty list to store DataFrames
    dfs = []

    # Iterate over rows in the original DataFrame
    for index, row in df.iterrows():
        timestamp = row['Timestamp']

        # Convert string of lists to actual lists
        chn_1 = ast.literal_eval(row['Channel 1'])
        chn_2 = ast.literal_eval(row['Channel 2'])
        chn_3 = ast.literal_eval(row['Channel 3'])
        chn_4 = ast.literal_eval(row['Channel 4'])

        # Create a DataFrame for the current row
        data = {
            'Timestamp': [timestamp] * len(chn_1),
            'Channel 1': chn_1,
            'Channel 2': chn_2,
            'Channel 3': chn_3,
            'Channel 4': chn_4,
        }
        df_row = pd.DataFrame(data)

        # Append the DataFrame to the list
        dfs.append(df_row)

    # Concatenate all DataFrames in the list
    expanded_df = pd.concat(dfs, ignore_index=True)

    return expanded_df

if __name__ == '__main__':
    # Replace with the actual CSV file path
    csv_filename = r"interpolated_file_signal.csv"
    
    # Read the CSV file into a pandas DataFrame
    original_df = pd.read_csv(csv_filename)

    # Expand the 'Chn' data into individual columns
    expanded_df = expand_chn_data(original_df)

    # Save the expanded DataFrame to a new CSV file
    expanded_csv_filename = r"C:\Users\Amarjith CK\Downloads\modified-matlab-code\expanded_output_signal.csv"
    expanded_df.to_csv(expanded_csv_filename, index=False)

    print(f"Expanded data saved to {expanded_csv_filename}")


In [ ]:
import pandas as pd

# Load data from the CSV file
file_path = 'expanded_output_signal.csv'  # Replace with the actual file path
df = pd.read_csv(file_path, parse_dates=['Timestamp'])
channels = ['Channel 1', 'Channel 2', 'Channel 3', 'Channel 4']

# Add 8000 to each channel value
df[channels] = df[channels] + 8000

# Save the modified data to a new CSV file
df.to_csv(r'C:\Users\Amarjith CK\Downloads\modified-matlab-code\modified_data.csv', index=False)


In [124]:
import time
time.monotonic()

1122930.39